In [10]:
cd Z:\###_PhD1_###\RebeccaBeveridge - P27 CdkCyclin Fdc1\CdkCyclin\ION_MOBILITY\CCSDs

Z:\###_PhD1_###\RebeccaBeveridge - P27 CdkCyclin Fdc1\CdkCyclin\ION_MOBILITY\CCSDs


In [11]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource,Column 
from bokeh.models import HoverTool
import seaborn as sns
import random
output_notebook()
%matplotlib inline
from IPython.display import HTML

def randomColorGenerator():
    colorGen = ''.join([random.choice('0123456789ABCDEF') for x in range(6)])
    return ''.join(["#",colorGen])

Loading BokehJS ...

In [12]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Interactive tools:
#### <font color='green'>Hover</font>  - hover over any data point and it will bring detailed information about partial atomic charge, standard error, atom ID
#### <font color='green'>Zoom</font> - drag on the plot to zoom on specific region of interest
#### <font color='green'>Legend</font> - click on legend marker to select/deselect datasets 

# p27-C-WT

In [13]:
# Import data
data = pd.read_csv('p27c.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]

hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS (Å²):", "$x{int} Å²"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=800, plot_height=750, x_range=(500,3000)
          , y_range=(0,1.05), title='p27-C-WT', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"

show(t)

# p27KID

In [14]:
# Import data
data = pd.read_csv('p27kid.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]

hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS (Å²):", "$x{int} Å²"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=800, plot_height=750, x_range=(500,3000)
          , y_range=(0,1.05), title='p27KID', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"


show(t)

# p27FL

In [15]:
# Import data
data = pd.read_csv('p27fl.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]

hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS (Å²):", "$x{int} Å²"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=800, plot_height=750, x_range=(500,8000)
          , y_range=(0,1.05), title='p27FL', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"

show(t)

# Cdk:Cyclin

In [16]:
# Import data
data = pd.read_csv('cdkcyclin.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]

hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS (Å²):", "$x{int} Å²"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=700, plot_height=600, x_range=(500,8000)
          , y_range=(0,1.05), title='Cdk:Cyclin', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"

show(t)

# Cdk:Cyclin:p27KID

In [17]:
# Import data
data = pd.read_csv('cdkcyclin_p27kid.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]


hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS (Å²):", "$x{int} Å²"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=700, plot_height=600, x_range=(500,8000)
          , y_range=(0,1.05), title='Cdk:Cyclin:p27KID', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"

show(t)

# Cdk:Cyclin:p27FL

In [19]:
# Import data
data = pd.read_csv('cdkcyclin_p27fl.csv', delimiter=',')
ccsData = data.CCS
intGlobal = data['global']
intCharge = data.loc[:, data.columns.str.startswith('z')]


hover = HoverTool(tooltips=[("Point #:", "$index"),
                           ("CCS:", "$x{int}"),
                           ("Normalized intensity:", "$y{0.00}"),
                           ("Charge state:", "@xlabel")])

TOOLS = [hover,"box_zoom,pan,wheel_zoom,reset,save"]
t = figure(tools=TOOLS, plot_width=700, plot_height=600, x_range=(500,8000)
          , y_range=(0,1.05), title='Cdk:Cyclin:p27FL', active_drag="box_zoom")

i = 0 
dataSource = []
for key in intCharge:
    legendKey = ''.join([key[1::],"+"])
    source = ColumnDataSource(data=dict(x=ccsData, y=intCharge[key], xlabel=([legendKey]*(len(ccsData)))))
    dataSource.append(source)
    t.line(x='x', y='y', line_color=randomColorGenerator(), line_width=4, alpha=0.7, legend=legendKey,
          source=dataSource[i]) 
    i=i+1
source = ColumnDataSource(data=dict(x=ccsData, y=intGlobal, xlabel=(['Global']*(len(ccsData)))))
dataSource.append(source)
t.line(x='x', y='y', line_color='#FF0080', line_width=4, alpha=0.7, legend="Global", source=dataSource[i])

t.title.text_font_size = "16pt"
t.xaxis.major_label_text_font_size = "12pt"
t.xaxis.axis_label = 'Collision Cross Section (Å²)'
t.xaxis.axis_label_text_font_size = "16pt"
t.yaxis.major_label_text_font_size = "12pt"
t.yaxis.axis_label = 'Normalized Intensity'
t.yaxis.axis_label_text_font_size = "16pt"
t.legend.click_policy="hide"

show(t)